In [ ]:
!pip install pyspark
!pip install findspark

In [2]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np

In [3]:
# Creating a spark context class
sc = SparkContext()

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("Saving and Loading a SparkML Model").getOrCreate()

In [5]:
#Read the csv file
df=spark.read.csv('/content/searchterms.csv', header=True)

In [6]:
df.toPandas().head()

,day,month,year,searchterm
0,12,11,2021,mobile 6 inch
1,12,11,2021,mobile latest
2,12,11,2021,tablet wifi
3,12,11,2021,laptop 14 inch
4,12,11,2021,mobile 5g


In [7]:
numRows = df.count()
print(numRows)

10000


In [8]:
numCols = len(df.columns)
print(numCols)

4


In [9]:
df.schema["searchterm"]

StructField('searchterm', StringType(), True)

In [10]:
wordCountsDF = (df
                .groupBy('searchterm').count())
wordCountsDF.show()

+-------------------+-----+
|         searchterm|count|
+-------------------+-----+
|          mobile 5g| 2301|
|ebooks data science|  410|
|      mobile 6 inch| 2312|
|     tablet 10 inch|  715|
|             laptop|  935|
|      mobile latest| 1327|
|      gaming laptop|  499|
|     laptop 14 inch|  461|
|        tablet wifi|  896|
|          pen drive|  144|
+-------------------+-----+



In [11]:
wordCountsDF.orderBy('count', ascending=False).toPandas().head()

,searchterm,count
0,mobile 6 inch,2312
1,mobile 5g,2301
2,mobile latest,1327
3,laptop,935
4,tablet wifi,896


In [13]:
# Unzip a trained sales forecasting model
!tar -xf /content/model.tar.gz

In [14]:
# You need LinearRegressionModel to load the model
from pyspark.ml.regression import LinearRegressionModel
model = LinearRegressionModel.load('sales_prediction.model')

In [17]:
# Using the sales forecast model, predict the sales for the year of 2023
from pyspark.ml.feature import VectorAssembler

def predict(year):
    assembler = VectorAssembler(inputCols=["year"],outputCol="features")
    data = [[year,0]]
    columns = ["year", "sales"]
    _ = spark.createDataFrame(data, columns)
    __ = assembler.transform(_).select('features','sales')
    predictions = model.transform(__)
    predictions.select('prediction').show()
predict(2023)

+------------------+
|        prediction|
+------------------+
|175.16564294006457|
+------------------+

